# Ricerca su grafi
Molti problemi possono essere modellati coem dei problemi di ricerca su grafi.

- **Stato iniziale**: stato da cui parte la ricerca.
- **Stato finale**: stato che si vuole raggiungere.
- **Stato corrente**: stato in cui si trova l'algoritmo.
- **Operatore**: azione che modifica lo stato corrente.
- **Cammino**: sequenza di stati che porta dallo stato iniziale a quello corrente.
- **Costo del cammino**: costo totale per raggiungere lo stato corrente.

Non tutte queste informazioni sono necessarie e/o disponibili per ogni problema.

Iniziamo modellando un grafo in modo esplicito. 
- Il grafo è rappresentabile con un dizionario di liste (di adiacenza).
- Le chiavi del dizionario esterno sono i nodi del grafo.
- Se il grafo non è orientato ogni arco è rappresentato due volte.

In [2]:
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B', 'G'],
    'E': ['B', 'G'],
    'F': ['C', 'G'],
    'G': ['D', 'E', 'F']
}

In questo primo esempio, il problema è quello di trovare un cammino che porti dal nodo $A$ al nodo $D$ (se esite) attraversando il minor numero di archi.

- **Stato iniziale**: $A$
- **Stato finale**: $D$
- **Stato corrente**: Il nodo in cui si trova l'algoritmo ad ogni passo.
- **Operatore**: Spostarsi su un nodo adiacente.
- **Costo del cammino**: Numero di archi attraversati.

Volendo attraversare il minor numero di archi, possiamo utilizzare una ricerca in ampiezza (Breadth-First).

In [ ]:
class Agent():
    def __init__(self, graph, start, goal):
        self.graph = graph
        self.goal = goal
        self.frontier = [[start]]

    def next_states(self, path):
        pass

    def is_goal(self, state):
        pass

    def bfs(self):
        pass

## L'algoritmo BFS

1. Se la frontiera è vuota, restituisci `None`
2. Prendi il primo cammino dalla frontiera.
3. Se l'ultimo stato del cammino è uno stato goal, restituisci il cammino.
4. Altrimenti, genera tutti i cammini ottenibili aggiungendo un nuovo stato alla fine del cammino (evitando cicli).
5. Aggiungi i nuovi cammini alla frontiera.
6. Ripeti dal passo (1).

In [71]:
class Agent():
    def __init__(self, graph, start, goal):
        self.graph = graph
        self.goal = goal
        self.frontier = [[start]]
        self.result = self.bfs_yield() # metodo con yield, altrimenti self.bfs() ritorna una singola lista

        # Flag per vedere se qualche cammino è stato trovato
        found = False

        # Iteriamo su tutti i cammini generati
        for cammino in self.result:
            found = True
            print("Path to goal found:")
            self.print_cammino(cammino)  # Stampa il cammino trovato

        if not found:
            print("No path to the goal was found.")


    def next_states(self, path):
        # il percorso e' una lista del tipo [nodo0, nodo1, nodo2]
        # va preso l'ultimo nodo con pop() e vanno verificati
        # i suoi vicini. Non vanno presi vicini gia' presenti
        # nel percorso per evitare cicli.
        node = path[-1]
        next_states = self.graph.get(node)
        return [x for x in next_states if not x in path]

    def print_cammino(self, cammino):
        for i in range(len(cammino)-1):
            print(f'{cammino[i]} -> ', end="")
        print(f'{cammino[-1]}')

    def print_frontiera(self):
        print("Frontiers: ")
        for i in range(len(self.frontier)):
            print(self.frontier[i])


    def is_goal(self, state):
        if state == self.goal:
            return True
        return False

    def bfs(self):
        while True:
            if not self.frontier:
                return None

            # prelevo una lista (cammino)
            cammino = self.frontier.pop(0)

            # prelevo l'ultimo nodo del cammino
            last_state = cammino[-1]
            #print("Last State: ", last_state)


            # verifico che l'ultimo nodo sia un goal
            if self.is_goal(last_state):
                return cammino

            # se non e' un goal, calcolo i possibili percorsi da quel nodo
            # genero poi i nuovi cammini
            next_states = self.next_states(cammino)
            for new_state in next_states:
                new_cammino = list(cammino)
                new_cammino.append(new_state)
                self.frontier.append(new_cammino)

            #self.print_frontiera()

    # Implementazione alternativa di bfs() con yield
    def bfs_yield(self):

        if not self.frontier:
            return None

        # prelevo una lista (cammino)
        cammino = self.frontier.pop(0)

        # anziche' prelevare l'ultimo nodo del cammino, faccio tutto in un'unica riga
        if self.is_goal(cammino[-1]):
            # yield permette di continuare la ricerca anche dopo aver trovato la soluzione, restituendo una lista di cammini di lunghezza crescente.
            # utilizzando return ci assicuriamo di terminale l'esecuzione trovato il percorso piu' breve
            yield cammino

        # NOTA BENE: se avessimo un "else" qui, una volta trovato il goal,
        # non verrebbero più cercati altri percorsi nonostante l'utilizzo
        # del "yield" (che permette di continuare la ricerca)

        next_paths = []
        for state in self.next_states(cammino):
                next_paths.append(cammino + [state])
        self.frontier += next_paths
        yield from self.bfs_yield()


In [72]:
agent_bfs = Agent(graph=graph, start='A', goal='D')


Path to goal found:
A -> B -> D
Path to goal found:
A -> B -> E -> G -> D
Path to goal found:
A -> C -> F -> G -> D
Path to goal found:
A -> C -> F -> G -> E -> B -> D


### Implementazioni alternative
- È possibile implementare lo stesso comportamento rappresentando il grafo come lista di archi?

- La ricerca in ampiezza puo essere implementata in modo iterativo?

## Considerazioni aggiuntive
Non sempre è possibile modellare un grafo in modo esplicito.

- Il numero di stati puo' essere infinito.
- Il grafo puo' essere troppo grande per essere memorizzato.
- Il grafo puo' essere dinamico
- L'adiacenza puo' essere determinata da una funzione.

La funzione `next_states` puo' quindi dover fare valutazioni meno banali.
- Facciamo quindi uso di *operatori di trasformazione di stato*
- Dato uno stato ed una azione, un operatore di trasformazione di stato restituisce un nuovo stato.

# Cannibali e missionari
Tre missionari e tre cannibali devono attraversare un fiume.
1. Missionari e cannibali sono inizialmente sulla sponda sinistra del fiume e devono raggiungere la sponda destra.
2. Per farlo, hanno a disposizione una barca che puo' contenere al massimo due persone.
3. La barca non puo' attraversare il fiume senza nessuno a bordo.
4. Il numero di cannibali non puo' mai essere superiore al numero di cannibali su una delle due sponde.

L'obiettivo e' quello di trovare una sequenza minima di azioni che permetta a tutti di attraversare il fiume.

In [ ]:
class Agent():
    def __init__(self):
        pass

    def is_valid(self, state):
        pass # controlla se lo stato rispetta le regole del problema

    def apply_move(self, state, move):
        pass # applica una mossa ad uno stato

    def next_states(self, state):
        pass # genera tutti gli stati raggiungibili da uno stato

    def is_goal(self, state):
        pass

    def bfs(self):
        pass # implementa la ricerca in ampiezza

Il primo problema che bisogna porsi in un problema di ricerca è come rappresentare lo stato. In questo caso rappresentiamo lo stato come un dizionario:

In [ ]:
start = {
    'L': {'m': 3, 'c': 3},
    'R': {'m': 0, 'c': 0},
    'boat': 'L'
}

goal = {
    'L': {'m': 0, 'c': 0},
    'R': {'m': 3, 'c': 3},
    'boat': 'R'
}

Rappresentare esplicitamente tutti gli stati possibili e le loro transizioni sarebbe troppo dispendioso. Definiamo quindi una lista di azioni possibili che, applicate ad uno stato, generano un nuovo stato.

Data la posizione della barca, le azioni possibili sono:
1. `{'m': 1, 'c': 0}`: spostare un missionario
2. `{'m': 0, 'c': 1}`: spostare un cannibale
3. `{'m': 1, 'c': 1}`: spostare un missionario e un cannibale
4. `{'m': 2, 'c': 0}`: spostare due missionari
5. `{'m': 0, 'c': 2}`: spostare due cannibali

Non tutte le azioni sono pero' sempre possibili o risultano in uno stato valido.


Necessitiamo di una funzione che ci permetta di verificare se uno stato e' valido secondo le regole del problema per rimuovere gli stati non validi dalla frontiera.

Uno stato e' valido se in ogni sponda il numero di missionari e' maggiore o uguale al numero di cannibali, oppure se non ci sono missionari su quella sponda (e non ci sono quantita' negative di persone).

In [ ]:
def is_valid(self, state):
    for side in ['L', 'R']:
        if state[side]['m'] < 0 or state[side]['c'] < 0:
            return False
        if state[side]['m'] > 0 and state[side]['c'] > state[side]['m']:
            return False
    return True

Dato uno stato e una mossa, dobbiamo generare un nuovo stato. Per prima cosa copiamo lo stato corrente. Cambiamo la posizione della barca, e rimuoviamo le persone dalla sponda di partenza e le aggiungiamo alla sponda di arrivo.

Uno stato e' rappresentato da un dizionario che tiene traccia dei missionari e dei cannibali.

In [ ]:
import copy

def apply_move(self, state, move):
    # la copia profonda serve a fare in modo che eventuali
    # oggetti interni all'oggetto attuale, vengano copiati
    # per contenuto e non per riferimento, come invece avviene
    # nelle copie normali
    new_state = copy.deepcopy(state)
    # applicando una mossa sposto la barca a sinistra o destra
    new_state['boat'] = 'L' if state['boat'] == 'R' else 'R'
    for person in ['m', 'c']:
        # la barca nel vecchio stato rappresenta il punto di partenza
        # la barca nel nuovo stato rappresenta il punto di arrivo
        # le persone vengono tolte dalla riva di partenza
        # e aggiunte alla riva d'arrivo
        new_state[state['boat']][person] -= move[person]
        new_state[new_state['boat']][person] += move[person]
    return new_state

Creiamo nuovi stati prendendo in considerazione tutte le azioni possibili. Solamente le azioni che non violano le regole del problema verranno sostituite.

In [ ]:
def next_states(self, state):
    next_states = []
    for move in self.moves:
        new_state = self.apply_move(state, move)
        if self.is_valid(new_state):
            next_states.append(new_state)
    return next_states

In [ ]:
# Implementazione alternativa di next_states
def next_states(self, state):
    return [new_state for move in self.moves if self.is_valid(new_state := self.apply_move(state, move))]

A questo punto, possiamo eseguire una ricerca in ampiezza per trovare una sequenza di stati che porti dallo stato iniziale a quello finale.

Questo codice e' identico a quello per il grafo esplicito perche' abbiamo raggiunto un livello di astrazione sufficientemente elevato.

In [ ]:
import copy
import matplotlib.pyplot as plt
import numpy as np
from itertools import pairwise


class Agent_2():
    def __init__(self):
        self.start = {'L': {'m': 3, 'c': 3},
                      'R': {'m': 0, 'c': 0},
                      'boat':  'L'}

        self.goal  = {'L': {'m': 0, 'c': 0},
                      'R': {'m': 3, 'c': 3},
                      'boat':  'R'}

        # devo generare tutte le mosse possibili come una lista di stati.
        self.moves = [{'m': i, 'c': j} for i in range(3) for j in range(3) if i + j > 0 and i + j <= 2]

        self.frontier = [[self.start]]

    def is_valid(self, state):
        for side in ['L', 'R']:
            if state[side]['m'] < 0 or state[side]['c'] < 0:
                return False
            if state[side]['m'] > 0 and state[side]['c'] > state[side]['m']:
                return False
            return True

    def apply_move(self, state, move):
        new_state = copy.deepcopy(state)
        new_state['boat'] = 'L' if state['boat'] == 'R' else 'R'
        for person in ['m', 'c']:
            new_state[state['boat']][person] -= move[person]
            new_state[new_state['boat']][person] += move[person]
        return new_state

    def next_states(self, state):
        next_states = []
        for move in self.moves:
            new_state = self.apply_move(state, move)
            if self.is_valid(new_state):
                next_states.append(new_state)
        return next_states

    def is_goal(self, state):
        return state == self.goal

    def bfs(self):
        while self.frontier:
            path = self.frontier.pop(0)
            if self.is_goal(path[-1]):
                yield path
            self.frontier.extend([path + [state] for state in self.next_states(path[-1]) if state not in path])


    def states_to_moves(self, path):
        for i,j in pairwise(path):
            for move in self.moves:
                if self.apply_move(i, move) == j:
                    yield str(move['m']) + ' missionaries and ' + str(move['c']) + ' cannibals from ' + i['boat'] + ' to ' + j['boat']
                    break

    def plot_states(self, path):
        offsets= {'L': -1, 'R': 1}
        coords = []
        for i, state in enumerate(path):
            for side in ['L', 'R']:
                offset = offsets[side]
                for person, person_type in [('m', 0), ('c', 1)]:
                    for _ in range(state[side][person]):
                        coords.append([offsets[side] + offset, -i, person_type])
                        offset += offsets[side]
        coords = np.array(coords)
        plt.scatter(*coords[coords[:, 2] == 0][:, :2].T, c='b', label='missionaries')
        plt.scatter(*coords[coords[:, 2] == 1][:, :2].T, c='r', label='cannibals')
        # add legend
        plt.legend(['missionaries', 'cannibals'], loc='lower left')
        plt.show()

    def solve(self):
        path = next(self.bfs())
        if path:
            print(*path, sep = '\n')
            print(*self.states_to_moves(path), sep='\n')
            self.plot_states(path)

: 

In [74]:
agent_2 = Agent_2()
agent_2.solve()

{'L': {'m': 3, 'c': 3}, 'R': {'m': 0, 'c': 0}, 'boat': 'L'}
{'L': {'m': 3, 'c': 1}, 'R': {'m': 0, 'c': 2}, 'boat': 'R'}
{'L': {'m': 3, 'c': 2}, 'R': {'m': 0, 'c': 1}, 'boat': 'L'}
{'L': {'m': 3, 'c': 0}, 'R': {'m': 0, 'c': 3}, 'boat': 'R'}
{'L': {'m': 3, 'c': 1}, 'R': {'m': 0, 'c': 2}, 'boat': 'L'}
{'L': {'m': 2, 'c': 0}, 'R': {'m': 1, 'c': 3}, 'boat': 'R'}
{'L': {'m': 2, 'c': 1}, 'R': {'m': 1, 'c': 2}, 'boat': 'L'}
{'L': {'m': 1, 'c': 0}, 'R': {'m': 2, 'c': 3}, 'boat': 'R'}
{'L': {'m': 1, 'c': 1}, 'R': {'m': 2, 'c': 2}, 'boat': 'L'}
{'L': {'m': 0, 'c': 0}, 'R': {'m': 3, 'c': 3}, 'boat': 'R'}


TypeError: 'module' object is not callable